In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [2]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

runBlocking {
    val channel = Channel<Int>()
    val controlChannel = Channel<Int>()
    launch { // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) { delay(400); channel.send(2 * x); }
        controlChannel.send(0) // 감독관한테 나 일 다했어
        println("sending even number done")
    }
    launch { // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) { delay(800); channel.send(2 * x - 1) }
        channel.close()
        controlChannel.send(1) // 감독관한테 나 일 다했어
        println("sending odd number done")
    }
    launch { // 감독관 작업?
        controlChannel.receive()
        controlChannel.receive()
        channel.close()
        controlChannel.close()
    }
    // here we print five received integers:
    for(x in 1..10) { println(channel.receive()) }
    println("Done!")
}

2
1
4
6
3
8
sending even number done
10
5
7
sending odd number done
9
Done!


In [3]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

runBlocking {
    val channel = Channel<Int>()
    
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..10) { delay(400); channel.send(x) }
        channel.close();
    }
    
    // here we print five received integers:
    for (x in channel) { println(x) }
    // for(x in 1..5) { println(channel.receive()) }
    println("Done!")
}

1
2
3
4
5
6
7
8
9
10
Done!


In [4]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

val channel = Channel<Int>()

runBlocking {
    
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..10) { channel.send(x) }
        channel.close();
    }
    
    // here we print five received integers:
    for (x in channel) { delay(400); println(x) }
    println("Done!")
}

1
2
3
4
5
6
7
8
9
10
Done!


In [5]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*
import kotlinx.coroutines.flow.*

// 채널 안에 메모리가 허락하는 한 무한정의 버퍼를 두어 데이터를 많이 쌓아둘 수 있음 
val channel = Channel<Int>( Channel.UNLIMITED )
val count = MutableStateFlow<Int>(0) // multi-thread safe한 카운터를 용도

runBlocking {
    
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..10) {
            // delay(100); // 주석을 처리했다 풀었다, 그리고 50 으로도 해보라
            channel.send(x)
            count.update{it -> it+1}
            println("count ${count.value}")
        }
        channel.close();
    }
    
    // here we print five received integers:
    for (x in channel) {
        /// delay(100); // 주석을 처리했다 풀었다, 그리고 50 으로도 해보라
        count.update{it -> it-1}
        println("received ${x}, channel has ${count.value} items")
    }
    // for(x in 1..5) { println(channel.receive()) }
    println("Done!")
}

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
received 1, channel has 9 items
received 2, channel has 8 items
received 3, channel has 7 items
received 4, channel has 6 items
received 5, channel has 5 items
received 6, channel has 4 items
received 7, channel has 3 items
received 8, channel has 2 items
received 9, channel has 1 items
received 10, channel has 0 items
Done!
